In [1]:
import numpy as np
import pandas as pd
import os as os

In [2]:
os.chdir('../')

In [3]:
%pylab inline
exec(open("./run_model.py").read())
exec(open("./get_signal_names.py").read())
exec(open("./portfolio_optimizer.py").read())
exec(open("./non_linear_signal.py").read())

import datetime
from pylab import rcParams

Populating the interactive namespace from numpy and matplotlib


In [4]:
sig_type = 'rank'
#rank
#norm
size_tag = '0_500'
#0_3000
#0_500
#500_3000
#use_pickle = True 


    

sig_dir =      '../afp_data/' + sig_type + '/' +  size_tag + '/rps_data/'
fac_data_dir=  '../afp_data/' + sig_type + '/' +  size_tag + '/factor_exposure_data/'
fac_ret_file = '../afp_data/' + sig_type + '/' +  size_tag + '/fac_ret_file.csv'

In [5]:
use_pickle = True 

if use_pickle :
    file_name = '../afp_pickle/'+'sig_'+sig_type +'_'+size_tag +'.pkl' 
    sig_df = pd.read_pickle(file_name)
    #sig_df = pd.read_pickle('../afp_pickle/_rank_0_500.pkl')
else :
    sig_df = pd.DataFrame()
    sig_files = os.listdir(sig_dir)
    sig_files = np.sort(sig_files)



    for sig_file in sig_files :
        if '.csv' not in sig_file :
            continue
        print(sig_file)
        this_file = sig_dir + sig_file
        this_df = pd.read_csv(this_file, index_col = 'date')
        this_df.index = [ datetime.datetime.strptime(val, '%Y-%m-%d') for val in this_df.index]
        #this_df = this_df.sort_values(['mve'], ascending=False)
        #this_df = this_df.iloc[start_stock_ind:end_stock_ind,:]
        sig_df = sig_df.append(this_df)
    #data = sig_df.copy()
    

In [6]:
begin_date = datetime.datetime.strptime('19900101', '%Y%m%d')
end_date = datetime.datetime.strptime('20170101', '%Y%m%d')

sig_df = sig_df[(sig_df.index>=begin_date) & (sig_df.index<end_date)]
sig_df = sig_df.fillna(sig_df.mean())

In [7]:
'res_ret' in sig_df.columns

True

In [8]:
#factor_names =  ['beta','mve', 'bm', 'roic', 'agr', 'mom12m', 'idiovol']
factor_names =  ['beta']
sig_names = get_signal_names()
ret = sig_df['ret']
sig_df = sig_df[sig_names]
sig_df = sig_df.drop(factor_names, axis=1)

In [9]:
'res_ret' in sig_df.columns

False

In [10]:

pm = PortfolioOptimizer(fac_data_dir=fac_data_dir,
                        fac_ret_file = fac_ret_file,
                        fac_cov_hl = 12.0,
                        fac_names = None
                       )



--------


In [11]:
#new_sig_df = pd.DataFrame()

#for sig_name in sig_df.columns :
#    this_sig = sig_df[sig_name]
#    this_sig = pm.neu_signal(this_sig)
#    this_sig = this_sig.fillna(this_sig.mean())
#    new_sig_df[sig_name] = this_sig
#sig_df = new_sig_df

In [12]:
np.where( sig_df.isnull().any() ) [0]

array([], dtype=int64)

In [ ]:
look_back =12
sample_decay = 0.5
num_sig_vec = range(5,80,5)

In [ ]:
signs_dict = get_signal_signs_2()
signs_vec = []

for i in   sig_df.columns :
    signs_vec.append(signs_dict[i])

glm_sig_df, sig_names = get_glmnet_sig(sig_df, ret, look_back = look_back,sample_decay =sample_decay,
                                       num_sig_vec =num_sig_vec, alpha = 0.5, signs_vec = signs_vec )



inside
1991-01-31 00:00:00
1991-02-28 00:00:00
1991-03-28 00:00:00
1991-04-30 00:00:00
1991-05-31 00:00:00
1991-06-28 00:00:00
1991-07-31 00:00:00
1991-08-30 00:00:00
1991-09-30 00:00:00
1991-10-31 00:00:00
1991-11-29 00:00:00
1991-12-31 00:00:00
1992-01-31 00:00:00
1992-02-28 00:00:00
1992-03-31 00:00:00
1992-04-30 00:00:00
1992-05-29 00:00:00
1992-06-30 00:00:00
1992-07-31 00:00:00
1992-08-31 00:00:00
1992-09-30 00:00:00
1992-10-30 00:00:00
1992-11-30 00:00:00
1992-12-31 00:00:00
1993-01-29 00:00:00
1993-02-26 00:00:00
1993-03-31 00:00:00
1993-04-30 00:00:00
1993-05-28 00:00:00
1993-06-30 00:00:00
1993-07-30 00:00:00
1993-08-31 00:00:00
1993-09-30 00:00:00
1993-10-29 00:00:00
1993-11-30 00:00:00
1993-12-31 00:00:00


In [ ]:
pnl_df = pd.DataFrame()
for i in range( glm_sig_df.shape[1]):
    print(i)
    glm_pnl_sr = pm.run_signal(sig_sr=glm_sig_df.iloc[:,i],neu_sig=True)
    pnl_df[num_sig_vec[i]] = glm_pnl_sr

In [ ]:
window = 12
#pnl_df_avg = ( pnl_df.rolling(window=window).mean()*sqrt(12) ) /( pnl_df.rolling(window=window).std() )
pnl_df_avg = ( pnl_df.rolling(window=window).mean()*sqrt(12) )    
max_ind = pnl_df_avg.idxmax(axis=1)
min_ind = pnl_df_avg.idxmin(axis=1)
max_ind_smt = max_ind.ewm(halflife=window).mean()

rcParams['figure.figsize'] = 20,10
max_ind_smt.plot()

In [ ]:
(pnl_df_avg.iloc[:,-1] - pnl_df_avg.iloc[:,0] ).plot()

In [ ]:
pnl_df.mean(axis=0)*sqrt(12)/pnl_df.std(axis=0)

In [ ]:
window = 24
tc = (0.001/12.0)

#pnl_df_avg = ( pnl_df.rolling(window=window).mean()*sqrt(12) ) /( pnl_df.rolling(window=window).std() )
pnl_df_avg = ( pnl_df.rolling(window=window).mean()*sqrt(12) )    
pnl_df_avg = pnl_df_avg - ( np.array(num_sig_vec)*tc )
max_ind = pnl_df_avg.idxmax(axis=1)
min_ind = pnl_df_avg.idxmin(axis=1)
max_ind_smt = max_ind.ewm(halflife=24).mean()

rcParams['figure.figsize'] = 20,10
max_ind_smt.plot()

In [ ]:
(pnl_df_avg.max(axis=1) - pnl_df_avg.iloc[:,0] ).plot()

In [ ]:
tc = (0.0005/12.0)
m_r = pnl_df.mean(axis=0) - np.array(num_sig_vec)*tc

In [ ]:
m_r.plot()

In [ ]:
pnl_df.cumsum().plot()